In [1]:
import os
os.chdir('../')
os.getcwd()

'd:\\Code Workspace\\Personal Project\\Personal Project - End to End Loan Status Classification'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
